In [3]:
import cv2
import numpy as np

In [4]:
#variable that will help us to resize the extracted handwritten digits
RESIZED_IMAGE_WIDTH = 30
RESIZED_IMAGE_HEIGHT= 40

In [5]:
#load classifiers on variables
digitClassifications = np.loadtxt("classifications.txt", np.float32)
flatDigitImages = np.loadtxt("flatDigitImages.txt", np.float32)
print(flatDigitImages)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
#convert digitClassifications into 1-d array (needed for KNN algorithm)
digitClassifications = digitClassifications.reshape((digitClassifications.size, 1))

In [7]:
#creates empty KNearest classifier.
knn = cv2.ml.KNearest_create()
KNearest = knn
knn.train(flatDigitImages, cv2.ml.ROW_SAMPLE, digitClassifications)

True

In [8]:
imgTestSample = cv2.imread("id dataset/test_image/2.png")
bf = cv2.bilateralFilter(imgTestSample, 50, 100, 100) #to remove noise
imgGray = cv2.cvtColor(bf, cv2.COLOR_BGR2GRAY)

In [9]:
retVal, th = cv2.threshold(imgGray, 150, 255, cv2.CHAIN_APPROX_NONE)
# cv2.imshow('test sample',th)
# cv2.waitKey(0)

In [40]:
# cv2.drawContours(imgTestSample, contours, -1, (0,255,0), 3)
# cv2.imshow('test sample',imgTestSample)
# cv2.waitKey(0)
# find the biggest countour (c) by the area
strFinalString = ""
contours, h = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnt = sorted(contours, key=cv2.contourArea)
c= cnt[0]
[intX, intY, intW, intH] = cv2.boundingRect(c)   
imgDigit = th[intY:intY+intH, intX:intX+intW]
cv2.imshow('test sample',imgDigit)
cv2.waitKey(0)


cntr, h = cv2.findContours(imgDigit, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
print(len(cntr))
for myc in cntr:
    [intX, intY, intW, intH] = cv2.boundingRect(myc)
    imgROI = imgDigit[intY:intY+intH, intX:intX+intW]
    print(imgROI.shape[0]*imgROI.shape[1])
    if(imgROI.shape[0]*imgROI.shape[1] > 100 and imgROI.shape[0]*imgROI.shape[1] < 1000):
        imgROIResized = cv2.resize(imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))

        cv2.imshow('test sample',imgROI)
        cv2.waitKey(0)

        finalResized = imgROIResized.reshape((1,RESIZED_IMAGE_WIDTH*RESIZED_IMAGE_HEIGHT))
        finalResized = np.float32(finalResized)

        retVal, results, resp, dist =KNearest.findNearest(finalResized, k=1)

        tmpString = str(int(results[0][0]))
        strFinalString = strFinalString + tmpString

print(strFinalString)   


88
128331
128331
2
299
322
460
391
9
299
437
391
506
414
368
483
330
414
368
299
462
460
440
66
400
462
462
418
330
25
484
484
418
440
30
418
483
484
506
40
396
30
88
176
374
154
176
88
198
198
418
374
264
110
264
110
110
198
308
160
391
300
506
272
440
240
286
180
462
306
440
255
374
140
396
208
462
342
264
220
352
198
460
256
460
289
322
180
460
33333333333333332222222222222222111111111111110000000000000000000000000000000
